# **LLM Routing with Apigee**

<table align="left">
    <td style="text-align: center">
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/apigee-samples/blob/main/llm-routing/llm_routing_v1.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo\"><br> Open in Colab
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fapigee-samples%2Fmain%2Fllm-routing%2Fllm_routing_v1.ipynb">
          <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
        </a>
      </td>    
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/apigee-samples/main/llm-routing/llm_routing_v1.ipynb">
          <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/llm-routing/llm_routing_v1.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
        </a>
      </td>
</table>
<br />
<br />
<br />

# Routing Sample

- This is a sample Apigee proxy to demonstrate the routing capabilities of Apigee across different LLM providers. In this sample we will use Google VertexAI, Mistral and HuggingFace as the LLM providers
- The framework will easily help onboarding other providers using configurations

![architecture](https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/llm-routing/images/arch.jpg?raw=1)

# Benefits of Routing with Apigee:

* **Configuration Driven Routing**: All the routing logic are driven through configuration which makes onboarding very easy
* **Security**: Irrespective of the model and providers, Apigee will secure the endpoints
* **Consistency**: Apigee can offer that layer of consistency to work with any LLM SDKs that are being used

## Setup

Use the following GCP CloudShell tutorial. Follow the instructions to deploy the sample.

[![Open in Cloud Shell](https://gstatic.com/cloudssh/images/open-btn.png)](https://ssh.cloud.google.com/cloudshell/open?cloudshell_git_repo=https://github.com/GoogleCloudPlatform/apigee-samples&cloudshell_git_branch=main&cloudshell_workspace=.&cloudshell_tutorial=llm-routing/docs/cloudshell-tutorial.md)

## Test Sample

### Install Dependencies

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install google-cloud-aiplatform
!pip install openai

### Authenticate your notebook environment (Colab only)
If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using Vertex AI Workbench or Colab Enterprise.

In [6]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Set the Variables

In [7]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
APIGEE_HOST="[your-apigee-host-domain]" # @param {type:"string"}
APIKEY="[your-apikey]" # @param {type:"string"}

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    raise ValueError("Please set your PROJECT_ID")
if not APIGEE_HOST or APIGEE_HOST == "[your-apigee-host-domain]":
    raise ValueError("Please set your APIGEE_HOST")
if not APIKEY or APIKEY == "[your-apikey]":
    raise ValueError("Please set your APIKEY")

API_ENDPOINT = "https://"+APIGEE_HOST+"/v1/samples/llm-routing/"
PROMPT="Suggest name for a flower shop"

### Select an LLM Provider

Select a provider from the dropdown. This will automatically set the model name used by the SDKs

Try picking different providers from the dropdown above. You will see that the same SDK is able to call the Apigee endpoint serving responses from different providers

In [22]:
import sys
from google.colab import auth

llm_provider = "select" # @param ["select","google", "huggingface", "mistral"]

if llm_provider == "google":
    model = "google/gemini-1.5-flash"
elif llm_provider == "mistral":
    model = "open-mistral-nemo"
elif llm_provider == "huggingface":
    model = "meta-llama/Llama-3.2-11B-Vision-Instruct"
else:
    raise ValueError("Invalid LLM provider")

### Using OpenAI SDK

In [ ]:
import openai

openai.api_key = APIKEY
openai.base_url = API_ENDPOINT
openai.default_headers = {"x-apikey": APIKEY, "x-llm-provider": llm_provider}

completion = openai.chat.completions.create(
    model=model,
    messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": PROMPT
        }
      ]
    }
  ]
)
print(f"Using the OpenAI SDK, fetching the response from \"{model}\" provided by \"{llm_provider}\"")
print("\n")
print(completion.choices[0].message.content)

### Using Langchain SDK

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=model,
    api_key=APIKEY,
    base_url=API_ENDPOINT,
    default_headers = {"x-apikey": APIKEY, "x-llm-provider": llm_provider}
)
messages = [
  {
    "role": "user",
    "content": [
      {
        "type": "text",
        "text": PROMPT
      }
    ]
  }
]
print(f"Using the Langchain SDK, fetching the response from \"{model}\" provided by \"{llm_provider}\"")
print("\n")
print(llm.invoke(messages).content)